In [103]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep 
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select
import time

In [169]:
url = "https://camiler.diyanet.gov.tr/Sayfalar/default.aspx?PageNumber=1"

In [170]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
page = requests.get(url=url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')


In [171]:
# get list of regions to scrap 
list_level_1 = [x.get_text() for x in soup.find_all("option")]
print(len(list_level_1))

82


In [172]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)


Looking for [chromedriver 83.0.4103.39 mac64] driver in cache 
File found in cache by path [/Users/kimmontalibet/.wdm/drivers/chromedriver/83.0.4103.39/mac64/chromedriver]


In [176]:
dict_keys = {}
df = pd.DataFrame()


# ids of dropdown menus level 1 and 2 
id_button_1 = "ctl00_ctl35_g_7c8222a9_e27f_4831_b393_65197bc417d3_drpIl"
id_button_2 = "ctl00_ctl35_g_7c8222a9_e27f_4831_b393_65197bc417d3_drpIlce"

for i in range(1, len(list_level_1)): 
#for i in range(1, 2): 
    print(i)
    select = Select(driver.find_element_by_id(id_button_1))
    select.select_by_value('{}'.format(i))
    time.sleep(5)
    select = Select(driver.find_element_by_id(id_button_2))
    all_options = [o.get_attribute('value') for o in select.options]
    print(all_options)
    all_options_text = [o.get_attribute('text') for o in select.options]
    dict_keys[str(i)] = all_options_text
    for j, x in enumerate(all_options[1:]):
        print("x", x)
        select = Select(driver.find_element_by_id(id_button_2))
        time.sleep(3)
        select.select_by_value(x)
        time.sleep(3)
        
        # get number of pages
        number_pages = len(driver.find_element_by_class_name("pagination").text.replace("\n", ""))
        
        # iterate over pages 
        for num_page in range(0, number_pages):
            driver.find_element_by_id("ctl00_ctl35_g_7c8222a9_e27f_4831_b393_65197bc417d3_rptrpager_ctl0{}_lnkPager".format(num_page)).click()
            time.sleep(3)
            list_address = [x.text for x in driver.find_elements_by_tag_name("td")]
            if len(list_address)>4: 
                list_1 = list_address[0::2][2:]
                list_2 = list_address[1::2][2:]
                df_temp = pd.DataFrame({"city": list_1, "address": list_2})
                df_temp["region"] = list_level_1[i]
                df_temp["district"] = all_options_text[j+1]
                df = pd.concat([df, df_temp])
            #else: 
                # do nothing because empty page
            

25
['Seçiniz', '309', '325', '310', '311', '312', '313', '322', '314', '326', '315', '316', '317', '327', '318', '324', '319', '320', '321', '323', '328']
x 309
x 325
x 310
x 311
x 312
x 313
x 322
x 314
x 326
x 315
x 316
x 317
x 327
x 318
x 324
x 319
x 320
x 321
x 323
x 328
26
['Seçiniz', '335', '336', '329', '338', '339', '337', '330', '340', '331', '341', '332', '333', '334', '342']
x 335
x 336
x 329
x 338
x 339
x 337
x 330
x 340
x 331
x 341
x 332
x 333
x 334
x 342
27
['Seçiniz', '343', '344', '350', '345', '351', '346', '348', '349', '347']
x 343
x 344
x 350
x 345
x 351
x 346
x 348
x 349
x 347
28
['Seçiniz', '352', '353', '364', '365', '354', '366', '355', '356', '358', '367', '359', '357', '362', '360', '361', '363']
x 352
x 353
x 364
x 365
x 354
x 366
x 355
x 356
x 358
x 367
x 359
x 357
x 362
x 360
x 361
x 363
29
['Seçiniz', '369', '372', '373', '368', '370', '371']
x 369
x 372
x 373
x 368
x 370
x 371
30
['Seçiniz', '374', '1110', '375', '376', '377']
x 374
x 1110
x 375
x 376
x 37

In [ ]:
df.to_csv("mosquees_turquie.csv", index = False)

In [200]:
len(df)

90637